In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time

from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')

import predict
import base
import shop_classify as sc

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

%load_ext autoreload
%autoreload 2

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#读取休假表
calenders = base.ReadCalendarCSV();
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();

In [3]:
%reload_ext autoreload

In [4]:
train_pay_counts = pd.read_csv("../data/predict/fill_predict_data.txt", sep="\t", encoding='UTF-8', index_col=0)

In [5]:
train_pay_counts.columns = train_pay_counts.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d"))

In [6]:
def calculateSingleHolidayStd(id, start_week, end_week):
    df = train_pay_counts.loc[id][np.array(range(start_week * 7, end_week * 7))].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    stds = {}
    means = {}
    deviates = {}
    for i in range(1, 6):
        if i != 4:
            temp_df = df[df['holiday'] == i];
            
            stds[i] = temp_df['count'].std()
            means[i] = temp_df['count'].mean()
            deviates[i] = (np.max(temp_df['count']) - np.min(temp_df['count'])) / np.max(temp_df['count'])
            
    return stds, means, deviates;

In [7]:
def calculateHolidayStdsData(start_week, end_week):
    shop_mean_dict = {}
    shop_std_dict = {}
    shop_deviate_dict = {}
    for id in range(1,2001):
        print(id)
        shop_std_dict[id], shop_mean_dict[id], shop_deviate_dict[id] = calculateSingleHolidayStd(id, start_week, end_week)

    return shop_std_dict, shop_mean_dict, shop_deviate_dict;

In [8]:
shop_std_dict, shop_mean_dict, shop_deviate_dict = calculateHolidayStdsData(0, 3)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
shop_std_df = pd.DataFrame(shop_std_dict)
shop_std_df = shop_std_df.T

shop_mean_df = pd.DataFrame(shop_mean_dict)
shop_mean_df = shop_mean_df.T


shop_deviate_df = pd.DataFrame(shop_deviate_dict)
shop_deviate_df = shop_deviate_df.T

In [10]:
std_means = []
mean_stds = []
mean_gaps = []
mean_deviates = []
for id in range(1, 2001):
    std_means.append(np.mean(shop_std_df.loc[id]))
    mean_stds.append(np.std(shop_mean_df.loc[id]))
    mean_deviates.append(np.mean(shop_deviate_df.loc[id]))
    mean_gaps.append((np.max(shop_mean_df.loc[id]) - np.min(shop_mean_df.loc[id])) / np.max(shop_mean_df.loc[id]))
    print("id=%d, std_mean=%f mean_std=%f"%(id, std_means[id-1], mean_gaps[id-1]))

id=1, std_mean=23.625482 mean_std=0.173541
id=2, std_mean=28.064412 mean_std=0.600728
id=3, std_mean=14.201351 mean_std=0.423132
id=4, std_mean=20.642809 mean_std=0.568476
id=5, std_mean=49.233605 mean_std=0.260108
id=6, std_mean=16.919928 mean_std=0.204294
id=7, std_mean=16.035688 mean_std=0.403947
id=8, std_mean=12.187574 mean_std=0.448276
id=9, std_mean=20.055693 mean_std=0.302857
id=10, std_mean=12.013227 mean_std=0.173633
id=11, std_mean=14.851799 mean_std=0.300448
id=12, std_mean=11.572016 mean_std=0.357884
id=13, std_mean=12.406922 mean_std=0.105866
id=14, std_mean=11.335656 mean_std=0.321608
id=15, std_mean=17.228565 mean_std=0.375210
id=16, std_mean=11.464599 mean_std=0.291193
id=17, std_mean=11.078690 mean_std=0.340000
id=18, std_mean=26.537261 mean_std=0.446622
id=19, std_mean=25.231085 mean_std=0.431912
id=20, std_mean=10.793145 mean_std=0.163136
id=21, std_mean=10.929224 mean_std=0.445153
id=22, std_mean=12.737218 mean_std=0.182361
id=23, std_mean=32.440481 mean_std=0.3058

In [13]:
last_week_df = train_pay_counts[np.array(range(2 * 7, 3 * 7))]

In [14]:
#last_week
last_week_trend_dict = {}
for id in range(1, 2001):
    
    df = last_week_df.loc[id].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    trends = []
    for date in df.index:
        trends.append((df.loc[date]['count'] - shop_mean_df.loc[id][df.loc[date]['holiday']]) / shop_mean_df.loc[id][df.loc[date]['holiday']])

    last_week_trend_dict[id] = trends

In [15]:
last_week_trend_df = pd.DataFrame(last_week_trend_dict).T

In [16]:
shop_std_mean_df = pd.DataFrame()
shop_std_mean_df['std_1'] = shop_std_df[1]
shop_std_mean_df['std_2'] = shop_std_df[2]
shop_std_mean_df['std_3'] = shop_std_df[3]
shop_std_mean_df['std_5'] = shop_std_df[5]
shop_std_mean_df['mean_std'] = mean_stds
shop_std_mean_df['mean_gap'] = mean_gaps
shop_std_mean_df['std_mean'] = std_means
shop_std_mean_df['deviate_1'] = shop_deviate_df[1]
shop_std_mean_df['deviate_2'] = shop_deviate_df[2]
shop_std_mean_df['deviate_3'] = shop_deviate_df[3]
shop_std_mean_df['deviate_5'] = shop_deviate_df[5]
shop_std_mean_df['mean_deviate'] = mean_deviates

shop_std_mean_df['trend_0'] = last_week_trend_df[0]
shop_std_mean_df['trend_1'] = last_week_trend_df[1]
shop_std_mean_df['trend_2'] = last_week_trend_df[2]
shop_std_mean_df['trend_3'] = last_week_trend_df[3]
shop_std_mean_df['trend_4'] = last_week_trend_df[4]
shop_std_mean_df['trend_5'] = last_week_trend_df[5]
shop_std_mean_df['trend_6'] = last_week_trend_df[6]

In [17]:
#shop_std_mean_df = shop_std_mean_df.set_index(np.array(range(1, 2001)))

In [18]:
shop_std_mean_df.to_csv("../data/predict/common/periodicity_v2.txt", sep='\t', index=True, encoding='UTF-8')

In [14]:
shop_std_mean_df

,std_1,std_2,std_3,std_5,mean_std,mean_gap,std_mean,deviate_1,deviate_2,deviate_3,deviate_5,mean_deviate
1,28.147340,13.503086,32.695565,19.035055,15.015593,0.160706,23.345262,0.325490,0.105058,0.234848,0.135231,0.200157
2,26.128007,26.083200,35.921210,24.110855,31.869490,0.617387,28.060818,0.649485,0.455357,0.403727,0.335664,0.461058
3,11.527832,18.475209,12.767145,14.571662,18.842557,0.423197,14.335462,0.465909,0.310680,0.196850,0.224000,0.299360
4,19.257584,21.221059,27.784888,13.428825,53.597595,0.580080,20.423089,0.513274,0.338843,0.232558,0.113122,0.299449
5,126.584431,14.422205,21.071308,40.501029,23.708112,0.260108,50.644743,0.997797,0.130233,0.206897,0.274914,0.402460
6,10.394929,13.012814,13.076697,31.214313,8.817140,0.196928,16.924688,0.290598,0.195489,0.214286,0.445255,0.286407
7,15.445505,6.110101,15.567059,24.576411,28.677828,0.388517,15.424769,0.316176,0.085714,0.169399,0.221154,0.198111
8,8.674938,1.732051,19.655364,15.099669,14.935536,0.448276,11.290505,0.395062,0.061224,0.339806,0.297030,0.273280
9,24.129198,5.686241,35.594943,15.885003,34.338685,0.302857,20.323846,0.291339,0.052885,0.243986,0.090323,0.169633
10,22.101830,2.886751,11.789826,11.015141,6.845077,0.173633,11.948387,0.568345,0.056180,0.232323,0.191304,0.262038


In [76]:
shop_std_mean_df

,std_1,std_2,std_3,std_5,mean_std,mean_gap,std_mean,deviate_1,deviate_2,deviate_3,deviate_5,mean_deviate
1,21.878591,18.248288,10.066446,18.330303,9.812779,0.099792,17.130907,0.272085,0.120438,0.073529,0.136364,0.150604
2,32.432587,24.583192,33.060551,18.175075,20.495709,0.420799,27.062851,0.716667,0.382609,0.403727,0.269231,0.443058
3,11.009913,5.686241,3.511885,8.082904,17.937850,0.414462,7.072735,0.375000,0.134146,0.058824,0.144330,0.178075
4,17.424121,17.785762,32.186954,1.527525,45.794529,0.502513,17.231091,0.436508,0.263158,0.279476,0.015000,0.248535
5,72.488996,108.448144,104.543452,104.710713,40.370508,0.455753,97.547826,0.576052,0.994872,0.994595,0.981043,0.886640
6,10.072465,28.307832,23.158872,35.795717,7.213522,0.207881,24.333721,0.395604,0.413223,0.401786,0.510949,0.430391
7,17.866678,14.730920,22.912878,58.286648,18.672231,0.265477,28.449281,0.363014,0.184397,0.245902,0.538462,0.332944
8,14.215868,15.534907,19.924859,18.903263,10.137236,0.278986,17.144724,0.444444,0.379747,0.336538,0.356436,0.379291
9,33.045423,43.508620,26.764404,26.501572,25.942779,0.209989,32.455005,0.331010,0.306338,0.171617,0.151235,0.240050
10,20.309425,17.691806,4.582576,12.529964,7.153280,0.200000,13.778443,0.540541,0.393258,0.118421,0.242718,0.323735
